In [1]:
"""
Isolation Forest 학습 스크립트 (정상 데이터 전용)
- 정상 데이터로만 학습
- 모델 및 스케일러 저장
- 불필요한 분석/시각화 제외
"""

import os
import json
import pickle
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler

# =====================================
# 설정
# =====================================

# 입력 파일
INPUT_FEATURES = 'data/processed/1118_features.csv'

# 출력 디렉토리
OUTPUT_DIR_MODELS = 'models'
OUTPUT_DIR_RESULTS = 'results/isolation_forest'

# Isolation Forest 파라미터
IF_PARAMS = {
    'contamination': 'auto',  # 자동 오염도 추정
    'n_estimators': 100,      # 트리 개수
    'max_samples': 'auto',    # 서브샘플 크기
    'random_state': 42,
    'n_jobs': -1,             # 모든 CPU 사용
    'verbose': 0
}

# 디렉토리 생성
os.makedirs(OUTPUT_DIR_MODELS, exist_ok=True)
os.makedirs(OUTPUT_DIR_RESULTS, exist_ok=True)

# =====================================
# 1. 데이터 로드
# =====================================
print("="*70)
print("📂 Loading Normal Data")
print("="*70)

df = pd.read_csv(INPUT_FEATURES)
print(f"✅ Loaded {len(df)} samples")
print(f"📏 Shape: {df.shape}")

# =====================================
# 2. 특징과 메타데이터 분리
# =====================================

# 메타데이터 컬럼 (윈도우 정보)
metadata_cols = ['window_id', 'start_time', 'end_time', 'n_samples']

# 특징 컬럼 추출
feature_cols = [col for col in df.columns if col not in metadata_cols]

# 특징 행렬
X = df[feature_cols].values

print(f"\n✅ Features: {len(feature_cols)}")
print(f"✅ Training samples: {X.shape}")

# 기본 통계
print(f"\n📊 Data Statistics:")
print(f"   Mean: {X.mean():.4f}")
print(f"   Std:  {X.std():.4f}")
print(f"   Min:  {X.min():.4f}")
print(f"   Max:  {X.max():.4f}")

# =====================================
# 3. 데이터 정규화
# =====================================
print("\n" + "="*70)
print("🔧 Data Normalization")
print("="*70)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print(f"✅ Data normalized using StandardScaler")
print(f"   Scaled mean: {X_scaled.mean():.6f} (should be ~0)")
print(f"   Scaled std:  {X_scaled.std():.6f} (should be ~1)")

# =====================================
# 4. Isolation Forest 학습
# =====================================
print("\n" + "="*70)
print("🌲 Training Isolation Forest")
print("="*70)

print(f"Parameters:")
for key, value in IF_PARAMS.items():
    print(f"  {key:20s}: {value}")
print()

model = IsolationForest(**IF_PARAMS)
model.fit(X_scaled)

print("✅ Model training completed")

# =====================================
# 5. 학습 데이터에 대한 기본 통계
# =====================================
print("\n" + "="*70)
print("📊 Training Statistics")
print("="*70)

# 이상 점수 계산
anomaly_scores = model.score_samples(X_scaled)
predictions = model.predict(X_scaled)

# 통계
n_normal = np.sum(predictions == 1)
n_anomaly = np.sum(predictions == -1)

print(f"Training data predictions:")
print(f"  Normal (1):  {n_normal:5d} ({n_normal/len(predictions)*100:.1f}%)")
print(f"  Anomaly (-1): {n_anomaly:5d} ({n_anomaly/len(predictions)*100:.1f}%)")
print(f"\nAnomaly scores:")
print(f"  Mean:   {anomaly_scores.mean():.6f}")
print(f"  Std:    {anomaly_scores.std():.6f}")
print(f"  Min:    {anomaly_scores.min():.6f}")
print(f"  Max:    {anomaly_scores.max():.6f}")
print(f"  Median: {np.median(anomaly_scores):.6f}")

# =====================================
# 6. 모델 저장
# =====================================
print("\n" + "="*70)
print("💾 Saving Model and Scaler")
print("="*70)

# 모델 저장
model_path = os.path.join(OUTPUT_DIR_MODELS, 'isolation_forest.joblib')
with open(model_path, 'wb') as f:
    pickle.dump(model, f)
print(f"✅ Model saved: {model_path}")

# 스케일러 저장
scaler_path = os.path.join(OUTPUT_DIR_MODELS, 'scaler_if.joblib')
with open(scaler_path, 'wb') as f:
    pickle.dump(scaler, f)
print(f"✅ Scaler saved: {scaler_path}")

# =====================================
# 7. 학습 정보 저장 (JSON)
# =====================================
training_info = {
    'model_type': 'Isolation Forest',
    'training_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'data_info': {
        'input_file': INPUT_FEATURES,
        'n_samples': int(len(X)),
        'n_features': int(len(feature_cols)),
        'feature_cols': feature_cols
    },
    'model_params': IF_PARAMS,
    'training_stats': {
        'normal_predictions': int(n_normal),
        'anomaly_predictions': int(n_anomaly),
        'anomaly_ratio': float(n_anomaly / len(predictions)),
        'score_mean': float(anomaly_scores.mean()),
        'score_std': float(anomaly_scores.std()),
        'score_min': float(anomaly_scores.min()),
        'score_max': float(anomaly_scores.max()),
        'score_median': float(np.median(anomaly_scores))
    },
    'scaler_info': {
        'type': 'StandardScaler',
        'feature_mean': scaler.mean_.tolist(),
        'feature_std': scaler.scale_.tolist()
    }
}

info_path = os.path.join(OUTPUT_DIR_RESULTS, 'training_info.json')
with open(info_path, 'w') as f:
    json.dump(training_info, f, indent=4)
print(f"✅ Training info saved: {info_path}")

# =====================================
# 8. 요약
# =====================================
print("\n" + "="*70)
print("✅ Training Complete")
print("="*70)
print(f"\n[Saved Files]")
print(f"  • Model:        {model_path}")
print(f"  • Scaler:       {scaler_path}")
print(f"  • Training info: {info_path}")
print(f"\n[Training Summary]")
print(f"  • Samples:      {len(X)}")
print(f"  • Features:     {len(feature_cols)}")
print(f"  • Normal pred:  {n_normal} ({n_normal/len(predictions)*100:.1f}%)")
print(f"  • Anomaly pred: {n_anomaly} ({n_anomaly/len(predictions)*100:.1f}%)")
print(f"\n[Next Steps]")
print(f"  1. Use the model to predict on new data")
print(f"  2. Collect anomaly data for evaluation")
print(f"  3. Fine-tune contamination parameter if needed")
print("\n" + "="*70)

📂 Loading Normal Data
✅ Loaded 703 samples
📏 Shape: (703, 80)

✅ Features: 77
✅ Training samples: (703, 77)

📊 Data Statistics:
   Mean: 6965.9519
   Std:  45002.2106
   Min:  -9.8511
   Max:  779501.1670

🔧 Data Normalization
✅ Data normalized using StandardScaler
   Scaled mean: 0.000000 (should be ~0)
   Scaled std:  1.000000 (should be ~1)

🌲 Training Isolation Forest
Parameters:
  contamination       : auto
  n_estimators        : 100
  max_samples         : auto
  random_state        : 42
  n_jobs              : -1
  verbose             : 0

✅ Model training completed

📊 Training Statistics
Training data predictions:
  Normal (1):    693 (98.6%)
  Anomaly (-1):    10 (1.4%)

Anomaly scores:
  Mean:   -0.432699
  Std:    0.025405
  Min:    -0.557497
  Max:    -0.376018
  Median: -0.430073

💾 Saving Model and Scaler
✅ Model saved: models\isolation_forest.joblib
✅ Scaler saved: models\scaler_if.joblib
✅ Training info saved: results/isolation_forest\training_info.json

✅ Training Com